## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.
D:\Users\yehch\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\yehch\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\yehch\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\yehch\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

data_ratio = 20
# 將資料集減少為1/20
# 資料前處理 - X 標準化
x_train = preproc_x(x_train[::data_ratio])
x_test = preproc_x(x_test[::data_ratio])

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train[::data_ratio])
y_test = preproc_y(y_test[::data_ratio])

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])    
    return model

In [6]:
LEARNING_RATE = 1e-3
EPOCHS = 25 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [7]:
# 載入 Callbacks
'''
請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果
'''
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.2, 0.5]
redice_lr_patient = [3, 5]

In [8]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i+1, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    reduce_lr = ReduceLROnPlateau(factor=reduce_factor, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%i" % (i+1))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.20, reduce_patient: 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalizatio

2500/2500 [==============================] - 0s 92us/step - loss: 2.0748 - acc: 0.2892 - val_loss: 2.5246 - val_acc: 0.2200
Epoch 14/25
2500/2500 [==============================] - 0s 92us/step - loss: 2.0322 - acc: 0.3072 - val_loss: 2.4923 - val_acc: 0.2200
Epoch 15/25
2500/2500 [==============================] - 0s 90us/step - loss: 1.9819 - acc: 0.3228 - val_loss: 2.4642 - val_acc: 0.2140
Epoch 16/25
2500/2500 [==============================] - 0s 92us/step - loss: 1.9365 - acc: 0.3440 - val_loss: 2.4410 - val_acc: 0.2100
Epoch 17/25
2500/2500 [==============================] - 0s 91us/step - loss: 1.8990 - acc: 0.3488 - val_loss: 2.4186 - val_acc: 0.2120
Epoch 18/25
2500/2500 [==============================] - 0s 90us/step - loss: 1.8630 - acc: 0.3644 - val_loss: 2.3993 - val_acc: 0.2200
Epoch 19/25
2500/2500 [==============================] - 0s 92us/step - loss: 1.8245 - acc: 0.3736 - val_loss: 2.3808 - val_acc: 0.2160
Epoch 20/25
2500/2500 [==============================] - 0s 

Train on 2500 samples, validate on 500 samples
Epoch 1/25
2500/2500 [==============================] - 1s 534us/step - loss: 3.0067 - acc: 0.1088 - val_loss: 3.0452 - val_acc: 0.1140
Epoch 2/25
2500/2500 [==============================] - 0s 93us/step - loss: 2.9980 - acc: 0.1072 - val_loss: 3.0245 - val_acc: 0.1180
Epoch 3/25
2500/2500 [==============================] - 0s 89us/step - loss: 2.9705 - acc: 0.1140 - val_loss: 2.9972 - val_acc: 0.1240
Epoch 4/25
2500/2500 [==============================] - 0s 93us/step - loss: 2.9352 - acc: 0.1108 - val_loss: 2.9687 - val_acc: 0.1220
Epoch 5/25
2500/2500 [==============================] - 0s 94us/step - loss: 2.8963 - acc: 0.1148 - val_loss: 2.9350 - val_acc: 0.1260
Epoch 6/25
2500/2500 [==============================] - 0s 103us/step - loss: 2.8567 - acc: 0.1200 - val_loss: 2.9015 - val_acc: 0.1300
Epoch 7/25
2500/2500 [==============================] - 0s 97us/step - loss: 2.8116 - acc: 0.1328 - val_loss: 2.8731 - val_acc: 0.1320
Epoch 

Epoch 24/25
2500/2500 [==============================] - 0s 98us/step - loss: 0.3515 - acc: 0.9612 - val_loss: 1.9367 - val_acc: 0.4200
Epoch 25/25
2500/2500 [==============================] - 0s 100us/step - loss: 0.3477 - acc: 0.9632 - val_loss: 1.9314 - val_acc: 0.4120
Numbers of exp: 5, reduce_factor: 0.20, reduce_patient: 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_16 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_nor

2500/2500 [==============================] - 0s 101us/step - loss: 1.2900 - acc: 0.5716 - val_loss: 2.1649 - val_acc: 0.2940
Epoch 13/25
2500/2500 [==============================] - 0s 98us/step - loss: 1.2340 - acc: 0.5960 - val_loss: 2.1526 - val_acc: 0.2960
Epoch 14/25
2500/2500 [==============================] - 0s 98us/step - loss: 1.1849 - acc: 0.6188 - val_loss: 2.1463 - val_acc: 0.3020
Epoch 15/25
2500/2500 [==============================] - 0s 102us/step - loss: 1.1436 - acc: 0.6368 - val_loss: 2.1304 - val_acc: 0.3080
Epoch 16/25
2500/2500 [==============================] - 0s 98us/step - loss: 1.1024 - acc: 0.6604 - val_loss: 2.1049 - val_acc: 0.3000
Epoch 17/25
2500/2500 [==============================] - 0s 101us/step - loss: 1.0650 - acc: 0.6796 - val_loss: 2.0834 - val_acc: 0.3020
Epoch 18/25
2500/2500 [==============================] - 0s 118us/step - loss: 1.0301 - acc: 0.6956 - val_loss: 2.0779 - val_acc: 0.3020
Epoch 19/25
2500/2500 [==============================] -

Train on 2500 samples, validate on 500 samples
Epoch 1/25
2500/2500 [==============================] - 2s 715us/step - loss: 2.7066 - acc: 0.1752 - val_loss: 14.1354 - val_acc: 0.1100
Epoch 2/25
2500/2500 [==============================] - 0s 95us/step - loss: 2.2033 - acc: 0.2972 - val_loss: 4.5589 - val_acc: 0.1580
Epoch 3/25
2500/2500 [==============================] - 0s 97us/step - loss: 1.8542 - acc: 0.3676 - val_loss: 2.3636 - val_acc: 0.2400
Epoch 4/25
2500/2500 [==============================] - 0s 95us/step - loss: 1.6122 - acc: 0.4432 - val_loss: 2.7518 - val_acc: 0.2180
Epoch 5/25
2500/2500 [==============================] - 0s 97us/step - loss: 1.4965 - acc: 0.4764 - val_loss: 2.3731 - val_acc: 0.2340
Epoch 6/25
2500/2500 [==============================] - 0s 113us/step - loss: 1.3532 - acc: 0.5348 - val_loss: 3.4447 - val_acc: 0.1840

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/25
2500/2500 [==============================] - 0s

2500/2500 [==============================] - 0s 93us/step - loss: 2.1152 - acc: 0.2704 - val_loss: 2.4399 - val_acc: 0.2160
Epoch 24/25
2500/2500 [==============================] - 0s 93us/step - loss: 2.0925 - acc: 0.2792 - val_loss: 2.4275 - val_acc: 0.2200
Epoch 25/25
2500/2500 [==============================] - 0s 96us/step - loss: 2.0776 - acc: 0.2800 - val_loss: 2.4247 - val_acc: 0.2200
Numbers of exp: 10, reduce_factor: 0.50, reduce_patient: 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_31 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (De

2500/2500 [==============================] - 0s 97us/step - loss: 2.4528 - acc: 0.2008 - val_loss: 2.6404 - val_acc: 0.1740
Epoch 12/25
2500/2500 [==============================] - 0s 96us/step - loss: 2.4163 - acc: 0.2116 - val_loss: 2.6227 - val_acc: 0.1800
Epoch 13/25
2500/2500 [==============================] - 0s 97us/step - loss: 2.3879 - acc: 0.2136 - val_loss: 2.6013 - val_acc: 0.1760
Epoch 14/25
2500/2500 [==============================] - 0s 96us/step - loss: 2.3563 - acc: 0.2232 - val_loss: 2.5871 - val_acc: 0.1800
Epoch 15/25
2500/2500 [==============================] - 0s 95us/step - loss: 2.3258 - acc: 0.2296 - val_loss: 2.5703 - val_acc: 0.1860
Epoch 16/25
2500/2500 [==============================] - 0s 94us/step - loss: 2.2992 - acc: 0.2340 - val_loss: 2.5589 - val_acc: 0.1920
Epoch 17/25
2500/2500 [==============================] - 0s 97us/step - loss: 2.2727 - acc: 0.2448 - val_loss: 2.5409 - val_acc: 0.1880
Epoch 18/25
2500/2500 [==============================] - 0s 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()